# Project 009 - Transfer Learning with RESNET50 -  Keras Implementation


In [9]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, resnet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [10]:
#(You can swap this out for your own dataset later.)
#Save CIFAR-10 images to folders

import os
import cv2
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

classes = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

def save_split(split_name, images, labels):
    base = f"cifar10_{split_name}"
    os.makedirs(base, exist_ok=True)

    for i, (img, lbl) in enumerate(zip(images, labels)):
        cls = classes[lbl[0]]
        cls_path = os.path.join(base, cls)
        os.makedirs(cls_path, exist_ok=True)
        cv2.imwrite(os.path.join(cls_path, f"{i}.png"), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

save_split("train", x_train, y_train)
save_split("test", x_test, y_test)

In [11]:
#Use a generator that automatically resizes to 224×224

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_gen.flow_from_directory(
    "cifar10_train",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    "cifar10_test",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [12]:
#Apply ResNet50 Preprocessing

x_train = resnet50.preprocess_input(x_train)
x_test = resnet50.preprocess_input(x_test)

In [13]:
#Load Pre-trained ResNet50 (without top layer)

base_model = ResNet50(
    weights="imagenet",
    include_top=False, #this is to help us change the 1000 classes from ImageNet for us to add our own
    input_shape=(224, 224, 3) #this defines the expected input to be feed into the ResNet model
)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

In [14]:
#Add Custom Classification Head

num_classes = 10   # <---- ADD THIS

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

In [15]:
#Compile the Model

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
#Train the Classification Head Only (Feature Extraction)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10
)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 183s 111ms/step - accuracy: 0.6197 - loss: 1.1142 - val_accuracy: 0.7724 - val_loss: 0.6464
Epoch 2/10
 193/1563 ━━━━━━━━━━━━━━━━━━━━ 2:01 89ms/step - accuracy: 0.7283 - loss: 0.7949

In [ ]:
#Fine-Tune the Deeper ResNet Layers (Optional but increases accuracy)

for layer in base_model.layers[-30:]:   # Unfreeze last 30 layers
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=5
)


In [ ]:
#Evaluate

loss, acc = model.evaluate(x_test, y_test)
print("Final Test Accuracy:", acc)

#Predict on an External Image

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_external(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)

    img_array = np.expand_dims(img_array, axis=0)
    img_array = resnet50.preprocess_input(img_array)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)

    print("Predicted Class:", predicted_class)

    plt.imshow(image.load_img(img_path))
    plt.title("Predicted: " + str(predicted_class))
    plt.axis("off")
    plt.show()

predict_external("014.png")
